# CineMatch – Streamlit User Interface

**Project:** CineMatch Movie Recommendation System  
**Component:** User Interface (Streamlit)  
**Student Name:** Bhupesh Bhatia

---
## Purpose of This Notebook

This notebook documents the **Streamlit-based user interface** developed for the CineMatch project.

⚠️ Streamlit apps are executed via terminal, not inside Jupyter Notebook.

Run using:
```bash
streamlit run app.py
```

## UI Technology Stack

- Streamlit for UI
- Custom Netflix-style CSS
- Pickle files for ML integration
- TMDB API for posters


In [ ]:
import streamlit as st
import pickle
import pandas as pd
import requests

# ============================================================
# PAGE CONFIG
# ============================================================
st.set_page_config(
    page_title="CineMatch", 
    layout="wide",
    initial_sidebar_state="expanded"
)

# ============================================================
# CUSTOM CSS FOR NETFLIX-STYLE UI
# ============================================================
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap');
    
    * {
        font-family: 'Inter', sans-serif;
    }
    
    /* Main background - Netflix black */
    .stApp {
        background: #141414;
    }
    
    /* Remove default padding */
    .block-container {
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    
    /* Title styling - Netflix style */
    h1 {
        color: #ffffff !important;
        font-weight: 800 !important;
        font-size: 3.5rem !important;
        letter-spacing: -0.5px !important;
        margin-bottom: 0.5rem !important;
    }
    
    h3 {
        color: #e5e5e5 !important;
        font-weight: 600 !important;
        font-size: 1.5rem !important;
    }
    
    h4 {
        color: #ffffff !important;
        font-weight: 700 !important;
        font-size: 1.2rem !important;
        margin-top: 2rem !important;
    }
    
    /* Sidebar - Dark with red accent */
    [data-testid="stSidebar"] {
        background: linear-gradient(180deg, #000000 0%, #1a0000 100%);
        border-right: 1px solid #2d0000;
    }
    
    [data-testid="stSidebar"] h1 {
        color: #e50914 !important;
        font-size: 2.8rem !important;
        font-weight: 900 !important;
        text-transform: uppercase;
        letter-spacing: 2px;
    }
    
    [data-testid="stSidebar"] p, [data-testid="stSidebar"] .stMarkdown {
        color: #b3b3b3 !important;
    }
    
    /* Radio buttons - Netflix style */
    [data-testid="stSidebar"] .stRadio > label {
        color: #ffffff !important;
        font-weight: 600 !important;
        font-size: 1.1rem !important;
    }
    
    [data-testid="stSidebar"] .stRadio > div {
        background: transparent !important;
    }
    
    [data-testid="stSidebar"] .stRadio label {
        color: #b3b3b3 !important;
        font-size: 1rem !important;
        padding: 8px 0 !important;
        transition: color 0.3s ease !important;
    }
    
    [data-testid="stSidebar"] .stRadio label:hover {
        color: #ffffff !important;
    }
    
    /* Button styling - Netflix red */
    .stButton>button {
        background: #e50914 !important;
        color: white !important;
        border: none !important;
        padding: 16px 32px !important;
        font-size: 1.2rem !important;
        font-weight: 700 !important;
        border-radius: 4px !important;
        transition: all 0.3s ease !important;
        text-transform: uppercase;
        letter-spacing: 1px;
    }
    
    .stButton>button:hover {
        background: #f40612 !important;
        transform: scale(1.05) !important;
        box-shadow: 0 8px 30px rgba(229, 9, 20, 0.5) !important;
    }
    
    /* Slider styling */
    .stSlider {
        padding: 1rem 0;
    }
    
    .stSlider > label {
        color: #ffffff !important;
        font-weight: 700 !important;
        font-size: 1rem !important;
    }
    
    .stSlider [data-testid="stThumbValue"] {
        color: #e50914 !important;
        font-weight: 700 !important;
    }
    
    /* Metric styling */
    [data-testid="stMetricValue"] {
        color: #e50914 !important;
        font-size: 2.5rem !important;
        font-weight: 800 !important;
    }
    
    [data-testid="stMetricLabel"] {
        color: #b3b3b3 !important;
        font-size: 0.9rem !important;
        text-transform: uppercase;
        letter-spacing: 1px;
    }
    
    /* Info box styling */
    .stAlert {
        background: rgba(229, 9, 20, 0.1) !important;
        border-left: 4px solid #e50914 !important;
        color: #ffffff !important;
        border-radius: 4px !important;
    }
    
    /* Selectbox styling */
    .stSelectbox label {
        color: #ffffff !important;
        font-weight: 700 !important;
        font-size: 1rem !important;
    }
    
    .stSelectbox > div > div {
        background: #2d2d2d !important;
        color: #ffffff !important;
        border: 1px solid #404040 !important;
        border-radius: 4px !important;
    }
    
    /* Caption styling */
    .stCaptionContainer {
        color: #808080 !important;
        font-size: 0.85rem !important;
    }
    
    /* Expander styling - Netflix card style */
    .streamlit-expanderHeader {
        background: #2d2d2d !important;
        color: #ffffff !important;
        border-radius: 4px !important;
        font-weight: 600 !important;
        border: 1px solid #404040 !important;
        transition: all 0.3s ease !important;
    }
    
    .streamlit-expanderHeader:hover {
        background: #3d3d3d !important;
        border-color: #e50914 !important;
    }
    
    .streamlit-expanderContent {
        background: #1a1a1a !important;
        border: 1px solid #404040 !important;
        border-top: none !important;
        color: #b3b3b3 !important;
    }
    
    /* Image container - Netflix card effect */
    img {
        border-radius: 6px;
        transition: all 0.3s ease;
        box-shadow: 0 4px 15px rgba(0,0,0,0.5);
    }
    
    img:hover {
        transform: scale(1.08);
        box-shadow: 0 8px 30px rgba(229, 9, 20, 0.4);
    }
    
    /* Category badge - Netflix style */
    .category-badge {
        display: inline-block;
        background: transparent;
        color: #b3b3b3;
        padding: 6px 14px;
        border-radius: 4px;
        font-size: 13px;
        margin: 4px;
        border: 1px solid #404040;
        font-weight: 600;
        transition: all 0.3s ease;
        cursor: pointer;
    }
    
    .category-badge:hover {
        background: #e50914;
        color: #ffffff;
        border-color: #e50914;
        transform: translateY(-2px);
    }
    
    /* Markdown text */
    .stMarkdown {
        color: #e5e5e5 !important;
    }
    
    /* Movie card container */
    .movie-card {
        background: #1a1a1a;
        border-radius: 8px;
        padding: 1rem;
        margin: 0.5rem 0;
        border: 1px solid #2d2d2d;
        transition: all 0.3s ease;
    }
    
    .movie-card:hover {
        background: #2d2d2d;
        border-color: #e50914;
        transform: translateY(-5px);
    }
    
    /* Match badge - Netflix style */
    .match-badge {
        background: linear-gradient(135deg, #e50914 0%, #b20710 100%);
        color: white;
        padding: 10px 16px;
        border-radius: 4px;
        text-align: center;
        font-weight: 800;
        font-size: 1rem;
        margin-top: 10px;
        box-shadow: 0 4px 15px rgba(229, 9, 20, 0.3);
        letter-spacing: 0.5px;
    }
    
    /* Divider */
    hr {
        border-color: #2d2d2d !important;
        margin: 2rem 0 !important;
    }
    
    /* Spinner */
    .stSpinner > div {
        border-top-color: #e50914 !important;
    }
    
    /* Section header glow effect */
    .section-header {
        color: #ffffff;
        font-size: 1.8rem;
        font-weight: 800;
        margin: 2rem 0 1rem 0;
        text-transform: uppercase;
        letter-spacing: 2px;
        border-left: 5px solid #e50914;
        padding-left: 1rem;
    }
</style>
""", unsafe_allow_html=True)

# ============================================================
# TMDB POSTER FETCH
# ============================================================
def fetch_poster(movie_id):
    try:
        response = requests.get(
            f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US",
            timeout=5
        )
        data = response.json()
        if 'poster_path' in data and data['poster_path']:
            return "https://image.tmdb.org/t/p/w500/" + data['poster_path']
        else:
            return "https://via.placeholder.com/500x750?text=No+Poster"
    except:
        return "https://via.placeholder.com/500x750?text=Error"

# ============================================================
# LOAD DATA
# ============================================================
@st.cache_data
def load_data():
    try:
        movies_data = pickle.load(open('movie_dict.pkl', 'rb'))
        similarity = pickle.load(open('similarity.pkl', 'rb'))
        
        if isinstance(movies_data, dict):
            movies = pd.DataFrame(movies_data)
        elif isinstance(movies_data, pd.DataFrame):
            movies = movies_data
        else:
            st.error(f"Error: Unexpected data type: {type(movies_data)}")
            st.stop()
        
        required_cols = ['movie_id', 'title', 'popularity_norm', 'recency_norm']
        missing_cols = [col for col in required_cols if col not in movies.columns]
        
        if missing_cols:
            st.error(f"Missing required columns: {missing_cols}")
            st.write(f"Available columns: {list(movies.columns)}")
            st.stop()
        
        return movies, similarity
        
    except FileNotFoundError:
        st.error("Error: movie_dict.pkl or similarity.pkl not found!")
        st.write("Please run the logic code first to generate the pickle files.")
        st.stop()
    except Exception as e:
        st.error(f"Error loading data: {e}")
        import traceback
        st.code(traceback.format_exc())
        st.stop()

movies, similarity = load_data()

# ============================================================
# RECOMMENDATION FUNCTION
# ============================================================
def recommend_with_control(movie, similarity_weight, popularity_weight, recency_weight):
    try:
        index = movies[movies['title'] == movie].index[0]
    except IndexError:
        st.error(f"Movie '{movie}' not found!")
        return [], [], []
    
    scores = []
    for i, sim_score in enumerate(similarity[index]):
        final_score = (
            similarity_weight * sim_score +
            popularity_weight * movies.iloc[i]['popularity_norm'] +
            recency_weight * movies.iloc[i]['recency_norm']
        )
        scores.append((i, final_score))
    
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:6]
    
    names = []
    posters = []
    explanations = []
    
    for i, score in scores:
        movie_id = movies.iloc[i]['movie_id']
        names.append(movies.iloc[i]['title'])
        posters.append(fetch_poster(movie_id))
        explanations.append({
            "Similarity": round(similarity[index][i], 2),
            "Popularity": round(movies.iloc[i]['popularity_norm'], 2),
            "Recency": round(movies.iloc[i]['recency_norm'], 2),
            "Final Score": round(score, 2)
        })
    
    return names, posters, explanations

# ============================================================
# SIDEBAR - NETFLIX STYLE
# ============================================================
with st.sidebar:
    st.markdown('<h1>CINEMATCH</h1>', unsafe_allow_html=True)
    st.markdown('<p style="color: #808080; font-size: 0.9rem; margin-top: -10px;">Your Premium Streaming Experience</p>', unsafe_allow_html=True)
    
    st.markdown("<br>", unsafe_allow_html=True)
    
    st.markdown("### BROWSE")
    nav_option = st.radio(
        "Navigation",
        ["Home", "My List", "Trending Now", "New Releases", "Popular"],
        label_visibility="collapsed"
    )
    
    st.markdown("<br>", unsafe_allow_html=True)
    
    st.markdown("### GENRES")
    categories = ["Action", "Comedy", "Drama", "Sci-Fi", "Thriller", "Romance", "Horror", "Documentary"]
    for i in range(0, len(categories), 2):
        col1, col2 = st.columns(2)
        with col1:
            st.markdown(f'<span class="category-badge">{categories[i]}</span>', unsafe_allow_html=True)
        if i + 1 < len(categories):
            with col2:
                st.markdown(f'<span class="category-badge">{categories[i+1]}</span>', unsafe_allow_html=True)
    
    st.markdown("<br><br>", unsafe_allow_html=True)
    
    st.markdown("### ACCOUNT")
    st.markdown("Switch Profiles")
    st.markdown("Manage Profiles")
    st.markdown("Account Settings")
    st.markdown("Help Center")
    st.markdown("Sign Out")
    
    st.markdown("<br>", unsafe_allow_html=True)
    
    col_m1, col_m2 = st.columns(2)
    with col_m1:
        st.metric("Movies", f"{len(movies):,}")
    with col_m2:
        st.metric("Users", "12.5K")

# ============================================================
# MAIN CONTENT
# ============================================================

# Hero section
st.markdown('<h1 style="margin-bottom: 0;">Discover Movies</h1>', unsafe_allow_html=True)
st.markdown('<p style="color: #b3b3b3; font-size: 1.2rem; margin-top: 0;">Personalized recommendations powered by advanced AI</p>', unsafe_allow_html=True)

st.markdown("<br>", unsafe_allow_html=True)

# Search and selection section
search_col, select_col = st.columns([1, 1])

with search_col:
    search_movie = st.selectbox(
        "Quick Search",
        movies['title'].values,
        key="search_bar"
    )

with select_col:
    selected_movie_name = st.selectbox(
        "Select Movie for Recommendations",
        movies['title'].values
    )

st.markdown("<br>", unsafe_allow_html=True)

# Advanced controls
st.markdown('<div class="section-header">CUSTOMIZE YOUR EXPERIENCE</div>', unsafe_allow_html=True)

col_a, col_b, col_c = st.columns(3)

with col_a:
    similarity_weight = st.slider("SIMILARITY MATCH", 0.0, 1.0, 0.7, 0.05)
    st.caption("Content similarity to your selection")

with col_b:
    popularity_weight = st.slider("POPULARITY INDEX", 0.0, 1.0, 0.2, 0.05)
    st.caption("Trending and popular titles")

with col_c:
    recency_weight = st.slider("RELEASE RECENCY", 0.0, 1.0, 0.1, 0.05)
    st.caption("Latest releases preference")

# Normalize weights
total = similarity_weight + popularity_weight + recency_weight
if total != 0:
    similarity_weight /= total
    popularity_weight /= total
    recency_weight /= total

st.info(f"ACTIVE WEIGHTS → Similarity: {similarity_weight:.0%} | Popularity: {popularity_weight:.0%} | Recency: {recency_weight:.0%}")

st.markdown("<br>", unsafe_allow_html=True)

# Main CTA button
if st.button("DISCOVER RECOMMENDATIONS", type="primary", use_container_width=True):
    with st.spinner("Analyzing your preferences..."):
        names, posters, explanations = recommend_with_control(
            selected_movie_name,
            similarity_weight,
            popularity_weight,
            recency_weight
        )
    
    if names:
        st.markdown("<br>", unsafe_allow_html=True)
        st.markdown('<div class="section-header">TOP PICKS FOR YOU</div>', unsafe_allow_html=True)
        
        cols = st.columns(5)
        
        for i in range(5):
            with cols[i]:
                st.image(posters[i], use_container_width=True)
                st.markdown(f'<p style="color: #ffffff; font-weight: 700; font-size: 1.1rem; margin: 10px 0 5px 0;">{names[i]}</p>', unsafe_allow_html=True)
                
                match_score = explanations[i]['Final Score']
                st.markdown(
                    f'<div class="match-badge">{match_score:.0%} MATCH</div>',
                    unsafe_allow_html=True
                )
                
                with st.expander("FULL ANALYSIS"):
                    st.markdown(f"**Similarity Score:** {explanations[i]['Similarity']:.0%}")
                    st.markdown(f"**Popularity Rating:** {explanations[i]['Popularity']:.0%}")
                    st.markdown(f"**Recency Factor:** {explanations[i]['Recency']:.0%}")
                    st.markdown(f"**Overall Match:** {explanations[i]['Final Score']:.0%}")
    else:
        st.warning("No recommendations available. Try another selection.")

# Footer
st.markdown("<br><br>", unsafe_allow_html=True)
st.markdown("---")
st.markdown('<p style="text-align: center; color: #808080; font-size: 0.9rem;">Powered by TMDB API | CineMatch Premium 2024</p>', unsafe_allow_html=True)